In [3]:
import numpy as np
from typing import Sequence

# Задачи оптимизации

## Введение<a name="introduce"></a>

В общем случае задача оптимизации является задачей поиска экстремума целевой функции в заданной области определения. Обычно в качестве целевой функции рассматривается гладкая функция $f\colon \mathbb R^n \to \mathbb R$, а в качестве области определения берётся замкнутое множество. Тогда в силу [теоремы Вейерштрасса](https://ru.wikipedia.org/wiki/Теорема_Вейерштрасса_о_функции_на_компакте) в заданной области достижим и минимум и максимум. При этом можно оптимизировать как минимум, так и максимум, однако чаще всего применяется оптимизация минимума. Одним из значительных применений задачи оптимизации является минимизация функции ошибки при обучении нейронных сетей.

## Целочисленная оптимизация<a name="integer"></a>

Однако, не всегда целевая функция является гладкой. Одним из подвидов задачи оптимизации является целочисленная или комбиноторная оптимизация. Рассмотрим несколько задач, решаемых этим методом.

### Задача о рюкзаке<a name="backpack"></a>

**Задача** Есть множество предметов, каждый из которых обладает двумя характеристиками: вес ($w$) и ценность ($v$). Необходимо выбрать подмножество из данных предметов, такое, чтобы суммарный вес выбранных предметов не превышал некоторое заданное число $w_{max}$, а суммарная ценность была максимальной.

Есть различные вариации формулировки задачи, но все они так или иначе сводятся к задаче оптимизации. Действительно, пусть у нас есть $n$ различных предметов. Тогда подмножество предметов можно записать как вектор в $n$-мерном пространстве, где $i$-я координата обозначает включается данный предмет в это подмножество или нет ($1$ и $0$ соответственно). Рассмотрим два вектора: $W = (w_0, w_1, \ldots, w_{n-1})$ и $V = (v_0, v_1, \ldots, v_{n-1})$ — веса и ценность предметов. Тогда задача сводится к оптимизации функции $f(X) = (X,V)$ с ограничением $(X, W) \leqslant w_{max}$. В некоторых формулировках задачи можно брать по несколько одинаковых предметов ($k$), таким образом дополнительным ограничением является $X \in [0, k]^n$, в нашем случае $X \in I^n$.

Для целочисленного решения этой задачи чаще всего применяется метод ветвей и границ. Но для начала попробуем решить эту задачу «в лоб», полным перебором. Для простоты воспользуемся рекурсивной функцией.

In [27]:
def brute_force(weights : Sequence[float], values : Sequence[float], max_weight : float) -> Sequence[float]:
    n = len(weights)
    assert len(values) == n
    optimal_x = np.zeros(n)
    optimal_value = 0
    
    def check(x : Sequence[float]):
        nonlocal optimal_x, optimal_value
        
        weight = sum(x[i] * weights[i] for i in range(n))
        if weight > max_weight:
            
            return
        
        value = sum((x[i] * values[i] for i in range(n)))
        if value > optimal_value:
            optimal_value = value
            optimal_x = np.array(x, copy=True)
            
    def step(x : Sequence[float] = np.empty(0), i : int = 0):
        if i == n:

            return check(x)
        
        step(np.append(x, 0), i+1)
        step(np.append(x, 1), i+1)
        
    step()
    
    return optimal_x

weights = np.array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
values = np.array([18, 20, 17, 19, 25, 21, 27, 23, 25, 24])
max_weight = 100

print(brute_force(weights, values, max_weight))

[1. 1. 0. 1. 1. 0. 1. 1. 1. 0.]


Собственно в приведённом решении перебираются все возможные подмножества предметов и для каждого проверяется, что вес набора не превосходит данный, а затем вычисляется ценность набора и сравнивается с предыдущим максимумом.

По факту мы строим дерево всех возможных решений. Это бинарное дерево высоты $n$, где в каждой вершине мы выбираем: барть нам этот предмет или нет, разветвляясь.

Первая оптимизация, которую мы можем выполнить, это отсечь те ветви, в которых вес уже превышает максимально допустимый. Кроме того, мы можем отсечь те ветви, в которых мы не сможем превысить имеющуюся на данный момент максимальную ценность, даже если возьмём все оставшиеся предметы.

In [29]:
def pack_a_backpack(weights : Sequence[float], values : Sequence[float], max_weight : float) -> Sequence[float]:
    n = len(weights)
    assert len(values) == n
    optimal_x = np.zeros(n)
    optimal_value = 0
    
    def step(x : Sequence[float] = np.empty(0), i : int = 0, current_weight : float = 0, available_value : float = sum(values)):
        nonlocal optimal_x, optimal_value

        if i == n:
            if available_value > optimal_value:
                optimal_x = np.array(x, copy=True)
                optimal_value = available_value
                
            return

        if available_value - values[i] > optimal_value:
            step(np.append(x, 0), i+1, current_weight, available_value - values[i])
            
        if current_weight + weights[i] <= max_weight:
            step(np.append(x, 1), i+1, current_weight + weights[i], current_value)
        
    step()
    
    return optimal_x

weights = np.array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
values = np.array([18, 20, 17, 19, 25, 21, 27, 23, 25, 24])
max_weight = 100

print(brute_force(weights, values, max_weight))

[1. 1. 0. 1. 1. 0. 1. 1. 1. 0.]


Итак мы получили решение данной задачи методом ветвей и границ (branch and bound algorithm). В общем виде этот алгоритм можно записать следующим образом:
```python
def step(...):
    if <условие завершения>:
        if <решение оптимальное>:
            <обновить оптимальное решение>
            
        return
    
    if <условие исключения>:
        step(с исключением)
        
    if <условие включения>:
        step(c включением)
```
Для закрепления, попробуем применить подобный подход к другой NP-полной задаче.

### Задача комивояжёра

**Задача** Есть множество городов, в которых комивояжёр должен побывать. Между городами есть различные пути сообщения с различными ценами за проезд. Как выбрать оптимальный маршрут так, чтобы побывать во всех городах и потратить минимальную сумму денег.

Данную задачу можно рассматривать как задачу поиска во взешанном графе гамильтонова цикла с минимальным весом.